# Auto Encoder over printer features

## Install required packages

In [47]:
if (!require("stringr")) { install.packages("stringr", lib='/R/library', repos='http://cran.us.r-project.org') }
if (!require("magrittr")) { install.packages("magrittr", lib='/R/library', repos='http://cran.us.r-project.org') }
if (!require("readr")) { install.packages("readr", lib='/R/library', repos='http://cran.us.r-project.org') }
if (!require("readr")) { install.packages("readr", lib='/R/library', repos='http://cran.us.r-project.org') }
if (!require("lubridate")) { install.packages("lubridate", lib='/R/library', repos='http://cran.us.r-project.org') }
if (!require("tidyr")) { install.packages("tidyr", lib='/R/library', repos='http://cran.us.r-project.org') }
if (!require("dplyr")) { install.packages("dplyr", lib='/R/library', repos='http://cran.us.r-project.org') }
if (!require("zoo")) { install.packages("zoo", lib='/R/library', repos='http://cran.us.r-project.org') }
if (!require("purrr")) { install.packages("purrr", lib='/R/library', repos='http://cran.us.r-project.org') }
if (!require("psych")) { install.packages("psych", lib='/R/library', repos='http://cran.us.r-project.org') }
if (!require("data.table")) { install.packages("data.table", lib='/R/library', repos='http://cran.us.r-project.org') }
if (!require("moments")) { install.packages("moments", lib='/R/library', repos='http://cran.us.r-project.org') }
if (!require("h2o")) { install.packages("h2o", type="source", lib='/R/library', repos=(c("http://h2o-release.s3.amazonaws.com/h2o/rel-turin/4/R"))) }
if (!require("autoencoder")) { install.packages("autoencoder", lib='/R/library', repos='http://cran.us.r-project.org') }
if (!require("IsolationForest")) { install.packages("IsolationForest", lib='/R/library', repos='http://cran.us.r-project.org') }
if (! ("methods" %in% rownames(installed.packages()))) { install.packages("methods", lib='/R/library', repos='http://cran.us.r-project.org') }
if (! ("statmod" %in% rownames(installed.packages()))) { install.packages("statmod", lib='/R/library', repos='http://cran.us.r-project.org') }
if (! ("stats" %in% rownames(installed.packages()))) { install.packages("stats", lib='/R/library', repos='http://cran.us.r-project.org') }
if (! ("graphics" %in% rownames(installed.packages()))) { install.packages("graphics", lib='/R/library', repos='http://cran.us.r-project.org') }
if (! ("RCurl" %in% rownames(installed.packages()))) { install.packages("RCurl", lib='/R/library', repos='http://cran.us.r-project.org') }
if (! ("jsonlite" %in% rownames(installed.packages()))) { install.packages("jsonlite", lib='/R/library', repos='http://cran.us.r-project.org') }
if (! ("tools" %in% rownames(installed.packages()))) { install.packages("tools", lib='/R/library', repos='http://cran.us.r-project.org') }
if (! ("utils" %in% rownames(installed.packages()))) { install.packages("utils", lib='/R/library', repos='http://cran.us.r-project.org') }

Loading required package: IsolationForest


IsolationForest 0.0-26 


## Load data

In [71]:
load('IndigoData.dat')
glimpse(data)

Observations: 1,211,693
Variables: 22
$ event_RowNumber (dbl) 15765585098, 15765585142, 15765585284, 15765585445,...
$ event_press     (int) 45000586, 45000586, 45000586, 45000586, 45000586, 4...
$ Product_Line    (fctr) HP Indigo 7600, HP Indigo 7600, HP Indigo 7600, HP...
$ series          (fctr) 7000 Family Sheet Fed Commercial Press, 7000 Famil...
$ Region          (fctr) North America, North America, North America, North...
$ SubRegion       (fctr) MidAtlantic United States, MidAtlantic United Stat...
$ District        (fctr) UNITED STATES, UNITED STATES, UNITED STATES, UNITE...
$ Ownership       (fctr) DIRECT, DIRECT, DIRECT, DIRECT, DIRECT, DIRECT, DI...
$ event_recNum    (int) 739828, 739878, 740037, 740221, 740224, 740554, 740...
$ event_date      (chr) "2016-06-08 00:00:00.000", "2016-06-08 00:00:00.000...
$ event_time      (int) 74703, 75903, 84709, 93603, 95930, 120644, 132213, ...
$ event_name      (fctr) SAMPLE_PIP_AND_IMO_PARAMETERS, SAMPLE_PIP_AND_IMO_...
$ event_state 

## Obtain only the necessary rows and columns from original dataset

In [72]:
relevant_data <- data %>%
    filter(event_state == "PRINT_STATE") %>%
    select(
        event_RowNumber, Product_Line, District, event_state, 
        PIP_Temperature, IO_temperature, IO_dirtiness, vessel_flow, 
        IO_Conductivity, CS_Voltage, Delta_Pressure, SWv
    )       

glimpse(relevant_data)

Observations: 203,675
Variables: 12
$ event_RowNumber (dbl) 15765585098, 15765585142, 15796104836, 15796104841,...
$ Product_Line    (fctr) HP Indigo 7600, HP Indigo 7600, HP Indigo 7600, HP...
$ District        (fctr) UNITED STATES, UNITED STATES, UNITED STATES, UNITE...
$ event_state     (fctr) PRINT_STATE, PRINT_STATE, PRINT_STATE, PRINT_STATE...
$ PIP_Temperature (int) 302, 298, 281, 271, 281, 281, 280, 284, 280, 278, 2...
$ IO_temperature  (int) 242, 243, 270, 257, 248, 248, 247, 245, 246, 257, 2...
$ IO_dirtiness    (int) 825, 825, 987, 987, 987, 987, 987, 987, 987, 987, 9...
$ vessel_flow     (int) 4092, 4170, 4164, 4056, 4530, 4164, 4110, 4254, 413...
$ IO_Conductivity (int) 4, 4, 5, 5, 4, 4, 4, 4, 4, 3, 4, 4, 4, 5, 4, 3, 3, ...
$ CS_Voltage      (int) 1846, 1846, 1954, 1954, 1954, 1954, 1954, 1954, 195...
$ Delta_Pressure  (int) 64, 64, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...
$ SWv             (fctr) V11_4_i40 I-Fix 1_i11, V11_4_i40 I-Fix 1_i11, V11_...


## Converting factor-based columns to one-hot-encoding

In [73]:
one_hot_product_line <- as.data.frame(dummy.code(relevant_data$Product_Line))
one_hot_district <- as.data.frame(dummy.code(relevant_data$District))
one_hot_event_state <- as.data.frame(dummy.code(relevant_data$event_state))
one_hot_swv <- as.data.frame(dummy.code(relevant_data$SWv))

## Treating missing values in vessel_flow and IO_Conductivity columns

In [74]:
presence_vessel_flow <- as.integer(!is.na(relevant_data$vessel_flow))
presence_io_conductivity <- as.integer(!is.na(relevant_data$IO_Conductivity))

## Binding new columns to new dataframe

In [75]:
numerical_data <- relevant_data %>%
    cbind(one_hot_product_line) %>%
    cbind(one_hot_district) %>%
    cbind(one_hot_event_state) %>%
    cbind(one_hot_swv) %>%
    mutate(p_vessel_flow = presence_vessel_flow) %>%
    mutate(p_io_conductivity = presence_io_conductivity) %>%
    select(-Product_Line, -District, -event_state, -SWv, -vessel_flow, -IO_Conductivity)

glimpse(numerical_data)

Observations: 203,675
Variables: 41
$ event_RowNumber        (dbl) 15765585098, 15765585142, 15796104836, 15796...
$ PIP_Temperature        (int) 302, 298, 281, 271, 281, 281, 280, 284, 280,...
$ IO_temperature         (int) 242, 243, 270, 257, 248, 248, 247, 245, 246,...
$ IO_dirtiness           (int) 825, 825, 987, 987, 987, 987, 987, 987, 987,...
$ CS_Voltage             (int) 1846, 1846, 1954, 1954, 1954, 1954, 1954, 19...
$ Delta_Pressure         (int) 64, 64, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
$ HP Indigo 7000         (dbl) 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
$ HP Indigo 7500         (dbl) 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
$ HP Indigo 7600         (dbl) 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...
$ BELGIUM                (dbl) 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
$ Canada                 (dbl) 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
$ FRANCE                 (dbl) 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
$ GERMANY       

## Training isolation forest

In [78]:
train_data <- numerical_data %>% select(-event_RowNumber)

In [79]:
trained_model <- IsolationTrees(train_data, rFactor = 0)

## Predicting with trained model

In [80]:
result <- AnomalyScore(train_data, trained_model)
scores <- result$outF

In [81]:
final_df <- cbind(numerical_data, scores)

In [82]:
head(final_df)

,event_RowNumber,PIP_Temperature,IO_temperature,IO_dirtiness,CS_Voltage,Delta_Pressure,HP Indigo 7000,HP Indigo 7500,HP Indigo 7600,BELGIUM,⋯,V11_0_i10,V11_2_i14,V11_2_i18,V11_4_i12,V11_4_i40,V11_4_i40 I-Fix 1_i11,V12_0_i22,p_vessel_flow,p_io_conductivity,scores
1,15765585098,302,242,825,1846,64,0,0,1,0,⋯,0,0,0,0,0,1,0,1,1,0.39139279696948
2,15765585142,298,243,825,1846,64,0,0,1,0,⋯,0,0,0,0,0,1,0,1,1,0.39139279696948
3,15796104836,281,270,987,1954,0,0,0,1,0,⋯,0,0,0,0,0,1,0,1,1,0.359296950483743
4,15796104841,271,257,987,1954,0,0,0,1,0,⋯,0,0,0,0,0,1,0,1,1,0.359296950483743
5,15796104862,281,248,987,1954,0,0,0,1,0,⋯,0,0,0,0,0,1,0,1,1,0.359296950483743
6,15796104929,281,248,987,1954,0,0,0,1,0,⋯,0,0,0,0,0,1,0,1,1,0.359296950483743


## List the most anomalous scores

In [84]:
most_anomalous <- final_df %>%
    filter(scores > 0.7) %>%
    arrange(desc(scores))

glimpse( most_anomalous )

Observations: 3,301
Variables: 42
$ event_RowNumber        (dbl) 14580766067, 14274659960, 14291599396, 15295...
$ PIP_Temperature        (int) 278, 286, 301, 287, 291, 275, 272, 282, 287,...
$ IO_temperature         (int) 285, 245, 265, 236, 232, 341, 282, 271, 266,...
$ IO_dirtiness           (int) 51, 987, 726, 678, 961, 757, 757, 1002, 1002...
$ CS_Voltage             (int) 2053, 1876, 2363, 1804, 2200, 2336, 2336, 17...
$ Delta_Pressure         (int) 0, 0, 64, 0, 64, 0, 0, 64, 64, 0, 0, 0, 0, 6...
$ HP Indigo 7000         (dbl) 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1,...
$ HP Indigo 7500         (dbl) 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,...
$ HP Indigo 7600         (dbl) 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,...
$ BELGIUM                (dbl) 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
$ Canada                 (dbl) 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
$ FRANCE                 (dbl) 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,...
$ GERMANY         

## Clustering anomaly scores

### Normalizing columns prior to clustering

In [ ]:
most_anomaloust <- normalized between 0 and 1 (all columns)

### Execute K-Means

In [68]:
score_clusters <- kmeans( select(highest_scores,-event_RowNumber,-scores) , iter.max = 5, centers = 5,  nstart=10)

In [69]:
score_clusters

K-means clustering with 5 clusters of sizes 112, 141, 13, 2868, 167

Cluster means:
       [,1]
1 0.7445592
2 0.7689815
3 0.8258511
4 0.7029967
5 0.7221235

Clustering vector:
   [1] 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 1 5 5 4 1 1 4 5 4 4 2 2 2 1 4
  [38] 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
  [75] 1 1 1 2 4 4 4 4 4 4 1 4 4 4 4 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 [112] 4 4 4 4 2 4 4 5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 4 4 4 4
 [149] 4 4 4 4 4 4 4 5 4 4 4 4 4 5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 4 4 4 4 4 4 4
 [186] 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2
 [223] 4 4 4 4 4 4 4 4 4 4 4 5 4 4 4 4 5 4 1 1 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 4 4
 [260] 4 4 5 5 4 5 1 1 4 5 4 4 4 4 4 1 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 2 5 4
 [297] 4 1 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 [334] 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 [371] 4 4 4 4